# Setup-1A

In [1]:
import os
import torch
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
from transformers import BertTokenizer, BertModel, BertConfig, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import transformers
transformers.logging.set_verbosity_error()

In [2]:
train_df = pd.read_csv('./Data/train.csv', sep='\t')
val_df = pd.read_csv('./Data/dev.csv', sep='\t')
train_df = train_df.dropna()
val_df = val_df.dropna()
print(len(train_df), len(val_df))

5708 1468


In [3]:
class TextSimilarityDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sentence1 = self.df.iloc[idx]['sentence1']
        sentence2 = self.df.iloc[idx]['sentence2']
        score = self.df.iloc[idx]['score']

        inputs = self.tokenizer.encode_plus(
            sentence1,
            sentence2,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'labels': torch.tensor(score, dtype=torch.float)
        }

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

train_dataset = TextSimilarityDataset(train_df, tokenizer, max_length=128)
val_dataset = TextSimilarityDataset(val_df, tokenizer, max_length=128)
print(len(train_dataset), len(val_dataset))

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
print(len(train_loader), len(val_loader))

5708 1468
90 23


In [4]:
import torch
import torch.nn as nn

class BertModel(nn.Module):
    def __init__(self, bert_model):
        super(BertModel, self).__init__()
        self.bert = bert_model
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs['pooler_output']
        logits = self.linear(pooled_output)
        return logits

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = BertModel(bert_model)
loaded_model.load_state_dict(torch.load('q1a_bert_model.pth'))
loaded_model = loaded_model.to(device)

In [5]:
def get_predictions(model, dataloader, device):
    model.eval()
    actual_scores = []
    predicted_scores = []
    model = model.to(device)
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask)
            actual_scores.extend(batch['labels'].numpy().tolist())
            predicted_scores.extend(outputs.flatten().tolist())
    return actual_scores, predicted_scores

val_actual, val_preds = get_predictions(loaded_model, val_loader, device)
val_pearson, _ = pearsonr(val_preds, val_actual)
print(f'Validation Pearson Correlation: {val_pearson}')

Validation Pearson Correlation: 0.8676728865750375


# Setup-1B

In [9]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

val_df = pd.read_csv('./Data/dev.csv', sep='\t')
val_df = val_df.dropna()
print(val_df.shape)

(1468, 3)


In [10]:
scaler = MinMaxScaler(feature_range=(0, 1))
val_df['score'] = scaler.fit_transform(val_df[['score']])

In [11]:
from sentence_transformers import SentenceTransformer, util
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer("all-MiniLM-L6-v2")

In [12]:
val_predicted_scores = []
    
for i in range(len(val_df)):
    s1 = val_df.iloc[i]['sentence1']
    s2 = val_df.iloc[i]['sentence2']
    emb1 = model.encode(s1, convert_to_tensor=True)
    emb2 = model.encode(s2, convert_to_tensor=True)
    cosine_sim = util.pytorch_cos_sim(emb1, emb2)
    val_predicted_scores.append(cosine_sim.item())

pearson_corr, _ = pearsonr(val_predicted_scores, val_df['score'])
print('Validation-Set Pearson Correlation:', pearson_corr)

Validation-Set Pearson Correlation: 0.8631423846336786


# Setup-1C

In [13]:
import os
import torch
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings("ignore")

In [15]:
val_df = pd.read_csv('./Data/dev.csv', sep='\t')
val_df = val_df.dropna()
print(val_df.shape)

(1468, 3)


In [16]:
scaler = MinMaxScaler(feature_range=(0, 1))
val_df['score'] = scaler.fit_transform(val_df[['score']])

In [18]:
from sentence_transformers import SentenceTransformer, util, SentencesDataset, InputExample, losses
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from torch.optim import Adam

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = SentenceTransformer("all-MiniLM-L6-v2")
loaded_model.load_state_dict(torch.load('q1c_sbert_model.pth'))
loaded_model = loaded_model.to(device)

In [19]:
def calculate_similarity_score(model, df):
    predicted = []
    for i in range(len(df)):
        emb1 = model.encode(df.iloc[i]['sentence1'], convert_to_tensor=True)
        emb2 = model.encode(df.iloc[i]['sentence2'], convert_to_tensor=True)
        cosine_sim = util.pytorch_cos_sim(emb1, emb2)
        predicted.append(cosine_sim.item())
    return predicted

val_predicted = calculate_similarity_score(loaded_model, val_df)
pearson_val, _ = pearsonr(val_predicted, val_df['score'])
print(f"Validation Pearson Score: {pearson_val}")

Validation Pearson Score: 0.8904479023415723
